In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
path = "~/Downloads/air.mon.mean.nc"

ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:    (lat: 73, level: 17, lon: 144, nbnds: 2, time: 486)
Coordinates:
  * level      (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2019-06-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) datetime64[ns] ...
    air        (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    CF-1.0
    title:          Monthly NCEP/DOE Reanalysis 2
    history:        created 2002/03 by Hoop (netCDF2.3)
    comments:       Data is from \nNCEP/DOE AMIP-II Reanalysis (Reanalysis-2)...
    platform:       Model
    source:         NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Model
    institution:    National Centers for Environmental Prediction
    dataset_title:  NCEP-DOE AMIP-II Reanalysis
    Re

In [3]:
y = ds['air']
air = y.loc['1979-01-01':'1999-12-01']
air

<xarray.DataArray 'air' (time: 252, level: 17, lat: 73, lon: 144)>
[45033408 values with dtype=float32]
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1999-12-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.40775 315.97388]
    units:                 degK
    precision:             2
    GRIB_id:               11
    GRIB_name:             TMP
    var_desc:              Air temperature
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Pressure Levels
    statistic:             Individual Obs
    parent_stat:           Othe

In [4]:
def clmMonTLLL(x):
    
    # Calculate the sizes of time dimension
    len_of_dim = x.sizes
    time_size = len_of_dim[x.dims[0]]
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((time_size % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
    
    # Get the sizes of latitude and longitude dimension
    lev_size = len_of_dim[x.dims[1]]
    lat_size = len_of_dim[x.dims[2]]
    lon_size = len_of_dim[x.dims[3]]
    
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[0]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    # Compute 12 months average
    ave_month = x.groupby(time_month).mean(time)
    
    # Copy and update the attributes
    ave_month.attrs = x.attrs
    ave_month = ave_month.rename("aveMonth")
    ave_month.attrs['time_op_ncl'] = "Climatology: " + str(int(time_size/no_of_months)) + " years"
    ave_month.attrs['info'] = "function clmMonTLLL"
    
    # Return the results
    return ave_month

In [5]:
xAve = clmMonTLLL(air)

In [6]:
xAve

<xarray.DataArray 'aveMonth' (month: 12, level: 17, lat: 73, lon: 144)>
array([[[[245.98235, ..., 245.98235],
         ...,
         [266.2162 , ..., 266.2162 ]],

        ...,

        [[212.21666, ..., 212.21666],
         ...,
         [249.37141, ..., 249.37141]]],


       ...,


       [[[247.39334, ..., 247.39334],
         ...,
         [266.2981 , ..., 266.2981 ]],

        ...,

        [[205.0924 , ..., 205.0924 ],
         ...,
         [252.49663, ..., 252.49663]]]], dtype=float32)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.4

In [7]:
def calcMonAnomTLLL(x, xAve):
    # x is the dataarray and xAve is the monthly averages array of x
    
    len_of_dim = x.sizes
    no_of_time = len_of_dim[x.dims[0]] # no_of_time = Size of time dimension
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((no_of_time % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[0]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    
    # Calculate anomalies by subtracting monthly means from actual dataarray
    xAnom = x.groupby(time_month) - xAve
        
    # Copy and update the attributes
    xAnom.attrs = x.attrs
    xAnom.attrs['anomaly_op_ncl'] = "Anomalies from Annual Cycle: calcMonAnomTLLL"
    
    # Drop the extra month co-ordinate from xAnom DataArray
    xAnom = xAnom.drop('month')
    xAnom = xAnom.rename('xAnomTLLL')
    
    # Return the new dataarray
    return xAnom

In [8]:
result = calcMonAnomTLLL(air, xAve)
result

<xarray.DataArray 'xAnomTLLL' (time: 252, level: 17, lat: 73, lon: 144)>
array([[[[-0.732346, ..., -0.732346],
         ...,
         [ 1.453796, ...,  1.453796]],

        ...,

        [[-2.21666 , ..., -2.21666 ],
         ...,
         [-4.22142 , ..., -4.22142 ]]],


       ...,


       [[[ 1.986664, ...,  1.986664],
         ...,
         [-2.778076, ..., -2.778076]],

        ...,

        [[-6.952393, ..., -6.952393],
         ...,
         [ 3.903366, ...,  3.903366]]]], dtype=float32)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1999-12-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_ran

In [9]:
result[0:60:5,14,14,14]

<xarray.DataArray 'xAnomTLLL' (time: 12)>
array([ 2.718094, -0.133316, -1.385239,  2.170013,  0.367142,  4.315247,
       -0.619553, -2.814774,  1.546646,  1.665222, -0.588089,  0.71666 ],
      dtype=float32)
Coordinates:
    level    float32 30.0
    lat      float32 55.0
    lon      float32 35.0
  * time     (time) datetime64[ns] 1979-01-01 1979-06-01 ... 1983-08-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.40775 315.97388]
    units:                 degK
    precision:             2
    GRIB_id:               11
    GRIB_name:             TMP
    var_desc:              Air temperature
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Pressure Levels
    statistic:             Individual Obs
    parent_stat:           Other
    standard_name:         air_temperature
    c